In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
import re
import json

In [36]:
merchantsData = """[{ name: "Walmart", country: "USA", city: "Bentonville AR", customers: "240M weekly", dailyCustomers: "34M", revenue: "$648B (2024)", valuation: "$534B", yearsInBusiness: 62, founded: 1962, sector: "Retail - Supermarket" },
    { name: "Amazon", country: "USA", city: "Seattle WA", customers: "310M active", dailyCustomers: "85M", revenue: "$575B (2023)", valuation: "$1.9T", yearsInBusiness: 30, founded: 1994, sector: "E-commerce/Tech" },
    { name: "Costco", country: "USA", city: "Issaquah WA", customers: "129M members", dailyCustomers: "3M", revenue: "$254B (2024)", valuation: "$387B", yearsInBusiness: 41, founded: 1983, sector: "Retail - Warehouse Club" },
    { name: "Target", country: "USA", city: "Minneapolis MN", customers: "200M annually", dailyCustomers: "5M", revenue: "$107B (2024)", valuation: "$68B", yearsInBusiness: 62, founded: 1962, sector: "Retail - Department Store" },
    { name: "The Home Depot", country: "USA", city: "Atlanta GA", customers: "40M per week", dailyCustomers: "5.7M", revenue: "$153B (2024)", valuation: "$384B", yearsInBusiness: 46, founded: 1978, sector: "Retail - Home Improvement" },
    { name: "Kroger", country: "USA", city: "Cincinnati OH", customers: "60M households", dailyCustomers: "11M", revenue: "$150B (2023)", valuation: "$41B", yearsInBusiness: 141, founded: 1883, sector: "Retail - Grocery" },
    { name: "Lowes", country: "USA", city: "Mooresville NC", customers: "18M per week", dailyCustomers: "2.6M", revenue: "$86B (2024)", valuation: "$132B", yearsInBusiness: 102, founded: 1921, sector: "Retail - Home Improvement" },
    { name: "CVS Health", country: "USA", city: "Woonsocket RI", customers: "5M daily", dailyCustomers: "5M", revenue: "$358B (2023)", valuation: "$75B", yearsInBusiness: 61, founded: 1963, sector: "Retail - Pharmacy" },
    { name: "Walgreens", country: "USA", city: "Deerfield IL", customers: "9M daily", dailyCustomers: "9M", revenue: "$139B (2023)", valuation: "$8B", yearsInBusiness: 123, founded: 1901, sector: "Retail - Pharmacy" },
    { name: "Best Buy", country: "USA", city: "Richfield MN", customers: "1.6M daily", dailyCustomers: "1.6M", revenue: "$46B (2024)", valuation: "$18B", yearsInBusiness: 58, founded: 1966, sector: "Retail - Electronics" },
    { name: "McDonalds", country: "USA", city: "Chicago IL", customers: "69M daily", dailyCustomers: "69M", revenue: "$25B (2023)", valuation: "$213B", yearsInBusiness: 69, founded: 1955, sector: "Fast Food" },
    { name: "Starbucks", country: "USA", city: "Seattle WA", customers: "100M weekly", dailyCustomers: "14M", revenue: "$36B (2023)", valuation: "$108B", yearsInBusiness: 53, founded: 1971, sector: "Coffee/Restaurant" },
    { name: "Subway", country: "USA", city: "Shelton CT", customers: "50M weekly", dailyCustomers: "7M", revenue: "$9.4B (2023)", valuation: "Private", yearsInBusiness: 59, founded: 1965, sector: "Fast Food" },
    { name: "Chick-fil-A", country: "USA", city: "Atlanta GA", customers: "8M daily", dailyCustomers: "8M", revenue: "$21B (2023)", valuation: "Private", yearsInBusiness: 78, founded: 1946, sector: "Fast Food" },
    { name: "Taco Bell", country: "USA", city: "Irvine CA", customers: "42M weekly", dailyCustomers: "6M", revenue: "$13B (2023)", valuation: "Part of Yum Brands", yearsInBusiness: 62, founded: 1962, sector: "Fast Food" },
    { name: "Burger King", country: "USA", city: "Miami FL", customers: "11M daily", dailyCustomers: "11M", revenue: "$1.8B (2023)", valuation: "Part of RBI $31B", yearsInBusiness: 70, founded: 1954, sector: "Fast Food" },
    { name: "Wendys", country: "USA", city: "Dublin OH", customers: "12M weekly", dailyCustomers: "1.7M", revenue: "$2.1B (2023)", valuation: "$3.5B", yearsInBusiness: 55, founded: 1969, sector: "Fast Food" },
    { name: "Dunkin", country: "USA", city: "Canton MA", customers: "3M daily", dailyCustomers: "3M", revenue: "$1.4B (2023)", valuation: "Part of Inspire Brands", yearsInBusiness: 74, founded: 1950, sector: "Coffee/Donuts" },
    { name: "Chipotle", country: "USA", city: "Newport Beach CA", customers: "1M daily", dailyCustomers: "1M", revenue: "$9.9B (2023)", valuation: "$75B", yearsInBusiness: 31, founded: 1993, sector: "Fast Casual" },
    { name: "Dominos Pizza", country: "USA", city: "Ann Arbor MI", customers: "3M daily", dailyCustomers: "3M", revenue: "$4.5B (2023)", valuation: "$14B", yearsInBusiness: 64, founded: 1960, sector: "Fast Food - Pizza" },
    { name: "Apple", country: "USA", city: "Cupertino CA", customers: "2B devices", dailyCustomers: "500K store visits", revenue: "$383B (2023)", valuation: "$3.4T", yearsInBusiness: 48, founded: 1976, sector: "Technology/Retail" },
    { name: "Microsoft", country: "USA", city: "Redmond WA", customers: "1.4B users", dailyCustomers: "N/A", revenue: "$211B (2023)", valuation: "$3.1T", yearsInBusiness: 49, founded: 1975, sector: "Technology/Software" },
    { name: "Google", country: "USA", city: "Mountain View CA", customers: "4B users", dailyCustomers: "3.5B searches", revenue: "$307B (2023)", valuation: "$2T", yearsInBusiness: 26, founded: 1998, sector: "Technology/Advertising" },
    { name: "eBay", country: "USA", city: "San Jose CA", customers: "132M active", dailyCustomers: "36M", revenue: "$10B (2023)", valuation: "$27B", yearsInBusiness: 29, founded: 1995, sector: "E-commerce" },
    { name: "PayPal", country: "USA", city: "San Jose CA", customers: "435M accounts", dailyCustomers: "48M transactions", revenue: "$30B (2023)", valuation: "$78B", yearsInBusiness: 25, founded: 1998, sector: "Fintech/Payments" },
    { name: "Etsy", country: "USA", city: "Brooklyn NY", customers: "96M active", dailyCustomers: "26M", revenue: "$2.7B (2023)", valuation: "$8B", yearsInBusiness: 19, founded: 2005, sector: "E-commerce - Handmade" },
    { name: "Shopify", country: "Canada", city: "Ottawa ON", customers: "4.4M merchants", dailyCustomers: "N/A", revenue: "$7.1B (2023)", valuation: "$95B", yearsInBusiness: 19, founded: 2004, sector: "E-commerce Platform" },
    { name: "Netflix", country: "USA", city: "Los Gatos CA", customers: "278M subscribers", dailyCustomers: "278M", revenue: "$33.7B (2023)", valuation: "$298B", yearsInBusiness: 27, founded: 1997, sector: "Streaming/Entertainment" },
    { name: "Spotify", country: "Sweden", city: "Stockholm", customers: "602M users", dailyCustomers: "165M", revenue: "$14B (2023)", valuation: "$53B", yearsInBusiness: 18, founded: 2006, sector: "Streaming/Music" },
    { name: "Uber", country: "USA", city: "San Francisco CA", customers: "150M users", dailyCustomers: "28M rides", revenue: "$37B (2023)", valuation: "$149B", yearsInBusiness: 15, founded: 2009, sector: "Technology/Transportation" },
    { name: "American Airlines", country: "USA", city: "Fort Worth TX", customers: "200M annually", dailyCustomers: "550K", revenue: "$53B (2023)", valuation: "$11B", yearsInBusiness: 98, founded: 1926, sector: "Airline" },
    { name: "Delta Air Lines", country: "USA", city: "Atlanta GA", customers: "200M annually", dailyCustomers: "550K", revenue: "$58B (2023)", valuation: "$32B", yearsInBusiness: 99, founded: 1925, sector: "Airline" },
    { name: "United Airlines", country: "USA", city: "Chicago IL", customers: "165M annually", dailyCustomers: "452K", revenue: "$53B (2023)", valuation: "$18B", yearsInBusiness: 98, founded: 1926, sector: "Airline" },
    { name: "Southwest Airlines", country: "USA", city: "Dallas TX", customers: "140M annually", dailyCustomers: "384K", revenue: "$26B (2023)", valuation: "$17B", yearsInBusiness: 53, founded: 1971, sector: "Airline" },
    { name: "Emirates", country: "UAE", city: "Dubai", customers: "50M annually", dailyCustomers: "137K", revenue: "$35B (2023)", valuation: "Private", yearsInBusiness: 38, founded: 1985, sector: "Airline" },
    { name: "Lufthansa", country: "Germany", city: "Cologne", customers: "130M annually", dailyCustomers: "356K", revenue: "$42B (2023)", valuation: "$8B", yearsInBusiness: 99, founded: 1953, sector: "Airline" },
    { name: "Air France-KLM", country: "France", city: "Paris", customers: "105M annually", dailyCustomers: "288K", revenue: "$32B (2023)", valuation: "$5B", yearsInBusiness: 20, founded: 2004, sector: "Airline" },
    { name: "British Airways", country: "UK", city: "London", customers: "43M annually", dailyCustomers: "118K", revenue: "$17B (2023)", valuation: "Part of IAG", yearsInBusiness: 104, founded: 1919, sector: "Airline" },
    { name: "Singapore Airlines", country: "Singapore", city: "Singapore", customers: "36M annually", dailyCustomers: "99K", revenue: "$16B (2023)", valuation: "$18B", yearsInBusiness: 76, founded: 1947, sector: "Airline" },
    { name: "Cathay Pacific", country: "Hong Kong", city: "Hong Kong", customers: "35M annually", dailyCustomers: "96K", revenue: "$10B (2023)", valuation: "$7B", yearsInBusiness: 77, founded: 1946, sector: "Airline" },
    { name: "Marriott International", country: "USA", city: "Bethesda MD", customers: "177M members", dailyCustomers: "1.5M rooms", revenue: "$23B (2023)", valuation: "$70B", yearsInBusiness: 97, founded: 1927, sector: "Hospitality/Hotels" },
    { name: "Hilton", country: "USA", city: "McLean VA", customers: "173M members", dailyCustomers: "1.2M rooms", revenue: "$10B (2023)", valuation: "$51B", yearsInBusiness: 105, founded: 1919, sector: "Hospitality/Hotels" },
    { name: "Hyatt", country: "USA", city: "Chicago IL", customers: "40M members", dailyCustomers: "300K rooms", revenue: "$6.6B (2023)", valuation: "$13B", yearsInBusiness: 66, founded: 1957, sector: "Hospitality/Hotels" },
    { name: "InterContinental Hotels", country: "UK", city: "Denham", customers: "100M members", dailyCustomers: "850K rooms", revenue: "$4.4B (2023)", valuation: "$17B", yearsInBusiness: 77, founded: 1946, sector: "Hospitality/Hotels" },
    { name: "Airbnb", country: "USA", city: "San Francisco CA", customers: "150M users", dailyCustomers: "2M bookings", revenue: "$10B (2023)", valuation: "$84B", yearsInBusiness: 16, founded: 2008, sector: "Hospitality/Vacation Rentals" },
    { name: "Booking.com", country: "Netherlands", city: "Amsterdam", customers: "1.5B room nights", dailyCustomers: "4M", revenue: "$21B (2023)", valuation: "Part of Booking Holdings $121B", yearsInBusiness: 27, founded: 1996, sector: "Travel/OTA" },
    { name: "Expedia", country: "USA", city: "Seattle WA", customers: "600M annually", dailyCustomers: "1.6M", revenue: "$12.8B (2023)", valuation: "$20B", yearsInBusiness: 28, founded: 1996, sector: "Travel/OTA" },
    { name: "Nike", country: "USA", city: "Beaverton OR", customers: "Not disclosed", dailyCustomers: "Not disclosed", revenue: "$51B (2023)", valuation: "$184B", yearsInBusiness: 60, founded: 1964, sector: "Apparel/Footwear" },
    { name: "Adidas", country: "Germany", city: "Herzogenaurach", customers: "Not disclosed", dailyCustomers: "Not disclosed", revenue: "$24B (2023)", valuation: "$39B", yearsInBusiness: 74, founded: 1949, sector: "Apparel/Footwear" },
    { name: "H&M", country: "Sweden", city: "Stockholm", customers: "2B transactions", dailyCustomers: "5.5M", revenue: "$22B (2023)", valuation: "$24B", yearsInBusiness: 78, founded: 1947, sector: "Retail - Fashion" },
    { name: "Zara", country: "Spain", city: "Arteixo", customers: "3M daily", dailyCustomers: "3M", revenue: "$35B (2023)", valuation: "$115B", yearsInBusiness: 49, founded: 1975, sector: "Retail - Fashion" },
    { name: "Uniqlo", country: "Japan", city: "Tokyo", customers: "1B annually", dailyCustomers: "2.7M", revenue: "$20B (2023)", valuation: "Part of Fast Retailing $68B", yearsInBusiness: 40, founded: 1984, sector: "Retail - Fashion" },
    { name: "Gap Inc", country: "USA", city: "San Francisco CA", customers: "75M annually", dailyCustomers: "205K", revenue: "$15B (2023)", valuation: "$8B", yearsInBusiness: 55, founded: 1969, sector: "Retail - Apparel" },
    { name: "Lululemon", country: "Canada", city: "Vancouver BC", customers: "25M annually", dailyCustomers: "69K", revenue: "$9.6B (2023)", valuation: "$45B", yearsInBusiness: 26, founded: 1998, sector: "Retail - Athletic Apparel" },
    { name: "TJX Companies", country: "USA", city: "Framingham MA", customers: "310M annually", dailyCustomers: "850K", revenue: "$54B (2024)", valuation: "$120B", yearsInBusiness: 48, founded: 1976, sector: "Retail - Off-Price" },
    { name: "Ross Stores", country: "USA", city: "Dublin CA", customers: "180M annually", dailyCustomers: "493K", revenue: "$20B (2023)", valuation: "$48B", yearsInBusiness: 42, founded: 1982, sector: "Retail - Off-Price" },
    { name: "Tesla", country: "USA", city: "Austin TX", customers: "6M vehicles sold", dailyCustomers: "5K deliveries", revenue: "$96B (2023)", valuation: "$1.1T", yearsInBusiness: 21, founded: 2003, sector: "Automotive/EV" },
    { name: "Toyota", country: "Japan", city: "Toyota City", customers: "10M annually", dailyCustomers: "27K", revenue: "$275B (2023)", valuation: "$264B", yearsInBusiness: 87, founded: 1937, sector: "Automotive" },
    { name: "Ford", country: "USA", city: "Dearborn MI", customers: "4M annually", dailyCustomers: "11K", revenue: "$176B (2023)", valuation: "$49B", yearsInBusiness: 121, founded: 1903, sector: "Automotive" },
    { name: "General Motors", country: "USA", city: "Detroit MI", customers: "5.9M annually", dailyCustomers: "16K", revenue: "$172B (2023)", valuation: "$54B", yearsInBusiness: 116, founded: 1908, sector: "Automotive" },
    { name: "Shell", country: "UK", city: "London", customers: "30M daily", dailyCustomers: "30M", revenue: "$316B (2023)", valuation: "$214B", yearsInBusiness: 117, founded: 1907, sector: "Energy/Gas Stations" },
    { name: "ExxonMobil", country: "USA", city: "Irving TX", customers: "25M daily", dailyCustomers: "25M", revenue: "$334B (2023)", valuation: "$510B", yearsInBusiness: 141, founded: 1882, sector: "Energy/Gas Stations" },
    { name: "BP", country: "UK", city: "London", customers: "11M daily", dailyCustomers: "11M", revenue: "$248B (2023)", valuation: "$105B", yearsInBusiness: 115, founded: 1909, sector: "Energy/Gas Stations" },
    { name: "Chevron", country: "USA", city: "San Ramon CA", customers: "8M daily", dailyCustomers: "8M", revenue: "$200B (2023)", valuation: "$285B", yearsInBusiness: 145, founded: 1879, sector: "Energy/Gas Stations" },
    { name: "Macys", country: "USA", city: "New York NY", customers: "33M annually", dailyCustomers: "90K", revenue: "$23B (2023)", valuation: "$4.5B", yearsInBusiness: 166, founded: 1858, sector: "Retail - Department Store" },
    { name: "Nordstrom", country: "USA", city: "Seattle WA", customers: "58M annually", dailyCustomers: "159K", revenue: "$15B (2023)", valuation: "$3B", yearsInBusiness: 123, founded: 1901, sector: "Retail - Department Store" },
    { name: "Kohls", country: "USA", city: "Menomonee Falls WI", customers: "43M annually", dailyCustomers: "118K", revenue: "$17B (2023)", valuation: "$2.5B", yearsInBusiness: 62, founded: 1962, sector: "Retail - Department Store" },
    { name: "Saks Fifth Avenue", country: "USA", city: "New York NY", customers: "12M annually", dailyCustomers: "33K", revenue: "$3B (est 2023)", valuation: "Private", yearsInBusiness: 97, founded: 1924, sector: "Retail - Luxury" },
    { name: "Neiman Marcus", country: "USA", city: "Dallas TX", customers: "7M annually", dailyCustomers: "19K", revenue: "$4.4B (2023)", valuation: "Private", yearsInBusiness: 117, founded: 1907, sector: "Retail - Luxury" },
    { name: "Bloomingdales", country: "USA", city: "New York NY", customers: "15M annually", dailyCustomers: "41K", revenue: "$3.2B (est 2023)", valuation: "Part of Macys", yearsInBusiness: 152, founded: 1872, sector: "Retail - Department Store" },
    { name: "Disney", country: "USA", city: "Burbank CA", customers: "145M parks visitors", dailyCustomers: "398K", revenue: "$88B (2023)", valuation: "$204B", yearsInBusiness: 101, founded: 1923, sector: "Entertainment/Media" },
    { name: "Universal Parks", country: "USA", city: "Orlando FL", customers: "50M annually", dailyCustomers: "137K", revenue: "$7.7B (2023)", valuation: "Part of Comcast", yearsInBusiness: 61, founded: 1964, sector: "Entertainment/Theme Parks" },
    { name: "AMC Theatres", country: "USA", city: "Leawood KS", customers: "300M annually", dailyCustomers: "822K", revenue: "$4.8B (2023)", valuation: "$2B", yearsInBusiness: 104, founded: 1920, sector: "Entertainment/Cinemas" },
    { name: "Regal Cinemas", country: "USA", city: "Knoxville TN", customers: "200M annually", dailyCustomers: "548K", revenue: "$3B (est 2023)", valuation: "Part of Cineworld", yearsInBusiness: 91, founded: 1989, sector: "Entertainment/Cinemas" },
    { name: "Ticketmaster", country: "USA", city: "Beverly Hills CA", customers: "500M tickets", dailyCustomers: "1.4M", revenue: "$17B (2023)", valuation: "Part of Live Nation", yearsInBusiness: 48, founded: 1976, sector: "Entertainment/Ticketing" },
    { name: "Verizon", country: "USA", city: "New York NY", customers: "144M subscribers", dailyCustomers: "N/A", revenue: "$134B (2023)", valuation: "$176B", yearsInBusiness: 24, founded: 2000, sector: "Telecommunications" },
    { name: "AT&T", country: "USA", city: "Dallas TX", customers: "240M subscribers", dailyCustomers: "N/A", revenue: "$122B (2023)", valuation: "$124B", yearsInBusiness: 139, founded: 1885, sector: "Telecommunications" },
    { name: "T-Mobile", country: "USA", city: "Bellevue WA", customers: "120M subscribers", dailyCustomers: "N/A", revenue: "$80B (2023)", valuation: "$233B", yearsInBusiness: 30, founded: 1994, sector: "Telecommunications" },
    { name: "Comcast", country: "USA", city: "Philadelphia PA", customers: "57M subscribers", dailyCustomers: "N/A", revenue: "$121B (2023)", valuation: "$170B", yearsInBusiness: 61, founded: 1963, sector: "Telecommunications/Cable" },
    { name: "Charter Communications", country: "USA", city: "Stamford CT", customers: "32M subscribers", dailyCustomers: "N/A", revenue: "$54B (2023)", valuation: "$60B", yearsInBusiness: 28, founded: 1993, sector: "Telecommunications/Cable" },
    { name: "Tesco", country: "UK", city: "Welwyn Garden City", customers: "21M weekly", dailyCustomers: "3M", revenue: "$86B (2023)", valuation: "$29B", yearsInBusiness: 105, founded: 1919, sector: "Retail - Grocery" },
    { name: "Carrefour", country: "France", city: "Boulogne-Billancourt", customers: "13M daily", dailyCustomers: "13M", revenue: "$98B (2023)", valuation: "$12B", yearsInBusiness: 63, founded: 1960, sector: "Retail - Grocery" },
    { name: "Aldi", country: "Germany", city: "Essen", customers: "100M weekly", dailyCustomers: "14M", revenue: "$134B (2023)", valuation: "Private", yearsInBusiness: 113, founded: 1913, sector: "Retail - Grocery" },
    { name: "Lidl", country: "Germany", city: "Neckarsulm", customers: "120M weekly", dailyCustomers: "17M", revenue: "$135B (2023)", valuation: "Private", yearsInBusiness: 94, founded: 1930, sector: "Retail - Grocery" },
    { name: "Whole Foods", country: "USA", city: "Austin TX", customers: "30M monthly", dailyCustomers: "1M", revenue: "$18B (2023)", valuation: "Part of Amazon", yearsInBusiness: 44, founded: 1980, sector: "Retail - Organic Grocery" },
    { name: "Trader Joes", country: "USA", city: "Monrovia CA", customers: "10M weekly", dailyCustomers: "1.4M", revenue: "$16B (est 2023)", valuation: "Private", yearsInBusiness: 56, founded: 1967, sector: "Retail - Grocery" },
    { name: "Publix", country: "USA", city: "Lakeland FL", customers: "1.3M daily", dailyCustomers: "1.3M", revenue: "$54B (2023)", valuation: "Private", yearsInBusiness: 94, founded: 1930, sector: "Retail - Grocery" },
    { name: "Safeway", country: "USA", city: "Pleasanton CA", customers: "600K daily", dailyCustomers: "600K", revenue: "$72B (2023)", valuation: "Part of Albertsons", yearsInBusiness: 116, founded: 1915, sector: "Retail - Grocery" },
    { name: "KFC", country: "USA", city: "Louisville KY", customers: "12M daily", dailyCustomers: "12M", revenue: "$28B (2023)", valuation: "Part of Yum Brands", yearsInBusiness: 83, founded: 1952, sector: "Fast Food" },
    { name: "Pizza Hut", country: "USA", city: "Plano TX", customers: "3M daily", dailyCustomers: "3M", revenue: "$13B (2023)", valuation: "Part of Yum Brands", yearsInBusiness: 66, founded: 1958, sector: "Fast Food - Pizza" },
    { name: "Papa Johns", country: "USA", city: "Louisville KY", customers: "2M weekly", dailyCustomers: "285K", revenue: "$2B (2023)", valuation: "$2.8B", yearsInBusiness: 40, founded: 1984, sector: "Fast Food - Pizza" },
    { name: "Five Guys", country: "USA", city: "Lorton VA", customers: "400K daily", dailyCustomers: "400K", revenue: "$2.1B (est 2023)", valuation: "Private", yearsInBusiness: 37, founded: 1986, sector: "Fast Food - Burgers" },
    { name: "Panera Bread", country: "USA", city: "Sunset Hills MO", customers: "400K daily", dailyCustomers: "400K", revenue: "$6B (2023)", valuation: "Private", yearsInBusiness: 42, founded: 1987, sector: "Fast Casual" },
    { name: "Shake Shack", country: "USA", city: "New York NY", customers: "100K daily", dailyCustomers: "100K", revenue: "$1B (2023)", valuation: "$3.5B", yearsInBusiness: 23, founded: 2001, sector: "Fast Casual - Burgers" },
    { name: "IKEA", country: "Sweden", city: "Almhult", customers: "900M annually", dailyCustomers: "2.5M", revenue: "$50B (2023)", valuation: "Private", yearsInBusiness: 81, founded: 1943, sector: "Retail - Furniture" },
    { name: "Wayfair", country: "USA", city: "Boston MA", customers: "22M active", dailyCustomers: "60K", revenue: "$12B (2023)", valuation: "$4B", yearsInBusiness: 22, founded: 2002, sector: "E-commerce - Home Goods" },
    { name: "Williams-Sonoma", country: "USA", city: "San Francisco CA", customers: "60M annually", dailyCustomers: "164K", revenue: "$8B (2023)", valuation: "$12B", yearsInBusiness: 69, founded: 1956, sector: "Retail - Home Goods" },
    { name: "Bed Bath & Beyond", country: "USA", city: "Union NJ", customers: "Bankrupt 2023", dailyCustomers: "N/A", revenue: "$7.9B (2022)", valuation: "Bankrupt", yearsInBusiness: 52, founded: 1971, sector: "Retail - Home Goods" }]"""

merchantsData = merchantsData.split("\n")
merchantsData = [x.replace('[','').replace("]","").replace(" },","}") for x in merchantsData]
data = pd.DataFrame()
for i, s in enumerate(merchantsData):
    s_fixed = re.sub(r'([{,]\s*)([a-zA-Z_][a-zA-Z0-9_]*)\s*:', r'\1"\2":', s)
    d = pd.DataFrame(json.loads(s_fixed), index=[len(data)])
    data = pd.concat([data, d], axis=0)

data

,name,country,city,customers,dailyCustomers,revenue,valuation,yearsInBusiness,founded,sector
0,Walmart,USA,Bentonville AR,240M weekly,34M,$648B (2024),$534B,62,1962,Retail - Supermarket
1,Amazon,USA,Seattle WA,310M active,85M,$575B (2023),$1.9T,30,1994,E-commerce/Tech
2,Costco,USA,Issaquah WA,129M members,3M,$254B (2024),$387B,41,1983,Retail - Warehouse Club
3,Target,USA,Minneapolis MN,200M annually,5M,$107B (2024),$68B,62,1962,Retail - Department Store
4,The Home Depot,USA,Atlanta GA,40M per week,5.7M,$153B (2024),$384B,46,1978,Retail - Home Improvement
...,...,...,...,...,...,...,...,...,...,...
93,Shake Shack,USA,New York NY,100K daily,100K,$1B (2023),$3.5B,23,2001,Fast Casual - Burgers
94,IKEA,Sweden,Almhult,900M annually,2.5M,$50B (2023),Private,81,1943,Retail - Furniture
95,Wayfair,USA,Boston MA,22M active,60K,$12B (2023),$4B,22,2002,E-commerce - Home Goods
96,Williams-Sonoma,USA,San Francisco CA,60M annually,164K,$8B (2023),$12B,69,1956,Retail - Home Goods


In [27]:
pd.DataFrame(d, index=[0])

,name,country,city,customers,dailyCustomers,revenue,valuation,yearsInBusiness,founded,sector
0,Walmart,USA,Bentonville AR,240M weekly,34M,$648B (2024),$534B,62,1962,Retail - Supermarket


In [41]:
merchants = data['name'].values

In [138]:
merchants

array(['Walmart', 'Amazon', 'Costco', 'Target', 'The Home Depot',
       'Kroger', 'Lowes', 'CVS Health', 'Walgreens', 'Best Buy',
       'McDonalds', 'Starbucks', 'Subway', 'Chick-fil-A', 'Taco Bell',
       'Burger King', 'Wendys', 'Dunkin', 'Chipotle', 'Dominos Pizza',
       'Apple', 'Microsoft', 'Google', 'eBay', 'PayPal', 'Etsy',
       'Shopify', 'Netflix', 'Spotify', 'Uber', 'American Airlines',
       'Delta Air Lines', 'United Airlines', 'Southwest Airlines',
       'Emirates', 'Lufthansa', 'Air France-KLM', 'British Airways',
       'Singapore Airlines', 'Cathay Pacific', 'Marriott International',
       'Hilton', 'Hyatt', 'InterContinental Hotels', 'Airbnb',
       'Booking.com', 'Expedia', 'Nike', 'Adidas', 'H&M', 'Zara',
       'Uniqlo', 'Gap Inc', 'Lululemon', 'TJX Companies', 'Ross Stores',
       'Tesla', 'Toyota', 'Ford', 'General Motors', 'Shell', 'ExxonMobil',
       'BP', 'Chevron', 'Macys', 'Nordstrom', 'Kohls',
       'Saks Fifth Avenue', 'Neiman Marcus', 'Bloomi

## Code Based

In [38]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI  # or use your local model wrapper
import pandas as pd
import time

In [39]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path = '/Users/vishaldawar/Phi-3-mini-4k-instruct-fp16.gguf',
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx = 2048,
    seed=42,
    verbose=False
)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_set_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_c4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f16                (n

In [40]:
# Initialize search tool
search = DuckDuckGoSearchRun()

# Initialize LLM — you can replace with any supported LangChain LLM
# llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.2)

In [42]:

# Define the prompt
prompt_template = """
You are a data research assistant. Given the merchant name below and search results,
extract structured factual information. Use only verified business facts (from official or reliable news/financial sources).

Merchant: {merchant_name}
Search Results:
{search_results}

Return a JSON object with the following fields (leave blank if not found):
{{
    "merchant_name": "{merchant_name}",
    "country": "",
    "city": "",
    "number_of_customers": "",
    "daily_customers": "",
    "yearly_revenue": "",
    "valuation": "",
    "years_in_business": "",
    "founded_year": "",
    "sector": "",
    "num_employees": ""
}}
"""

prompt = PromptTemplate(
    input_variables=["merchant_name", "search_results"],
    template=prompt_template
)

chain = LLMChain(llm=llm, prompt=prompt)


records = []

for i, merchant in enumerate(merchants):
    print(f"Searching for {i}, {merchant}...")
    # Step 1: Search
    query = f"{merchant} company profile number of customers revenue valuation headquarters site:linkedin.com OR site:crunchbase.com OR site:forbes.com OR site:reuters.com"
    search_results = search.run(query)

    # Step 2: Send to LLM for extraction
    response = chain.run(merchant_name=merchant, search_results=search_results)
    
    try:
        record = eval(response)  # response should be JSON-like
    except Exception:
        record = {"merchant_name": merchant, "raw_response": response}
    
    records.append(record)
    time.sleep(2)  # polite delay to avoid hammering APIs

# Convert to dataframe
df = pd.DataFrame(records)
df.to_csv("real_merchants_info.csv", index=False)
print("Data saved to real_merchants_info.csv")


/var/folders/dq/s07zv90x1qq7s275bwf08ylm0000gn/T/ipykernel_13302/3927879597.py:32: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
/Users/vishaldawar/venv/lib/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


🔍 Searching for 0, Walmart...


/var/folders/dq/s07zv90x1qq7s275bwf08ylm0000gn/T/ipykernel_13302/3927879597.py:44: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(merchant_name=merchant, search_results=search_results)


🔍 Searching for 1, Amazon...


/Users/vishaldawar/venv/lib/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


...

🔍 Searching for 96, Williams-Sonoma...


/Users/vishaldawar/venv/lib/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


🔍 Searching for 97, Bed Bath & Beyond...


/Users/vishaldawar/venv/lib/python3.9/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


✅ Data saved to real_merchants_info.csv


In [43]:
data = pd.read_csv("./real_merchants_info.csv")
data

,merchant_name,error,raw_response
0,Walmart,Parsing failed,"\nsolution: {\n ""merchant_name"": ""Walmart"",..."
1,Amazon,Parsing failed,"\nexplanation: {\n ""merchant_name"": ""Amazon..."
2,Costco,Parsing failed,"\n- bob: To provide accurate information, I'll..."
3,Target,Parsing failed,\n- [solution]: Since the provided search resu...
4,The Home Depot,Parsing failed,"\n**response:** {\n ""merchant_name"": ""The H..."
...,...,...,...
93,Shake Shack,Parsing failed,"\nresponse: {\n ""merchant_name"": ""Shake Sha..."
94,IKEA,Parsing failed,"\nOutput:\n{\n ""merchant_name"": ""IKEA"",\n ..."
95,Wayfair,Parsing failed,"\nTeacher: Given the available information, I'..."
96,Williams-Sonoma,Parsing failed,\n- [support]: Given the constraints of not ha...


In [113]:
from json_repair import repair_json

In [118]:
cleaned_jsons = []
for idx, row in data.iterrows():
    print(idx)
    text = row['raw_response']
    matches = re.findall(r'\{[^{}]*\}', text)

    if len(matches) == 0:
        cleaned = ""
    else:
        target_char = "//" # Example: remove content after a semicolon

        # Escape the target character if it's a special regex character
        escaped_target_char = re.escape(target_char)
        
        # Define the regex pattern to match the target character and everything after it on the same line
        pattern = fr'{escaped_target_char}.*'
        
        # Use re.sub with the re.M (multiline) flag to apply the pattern to each line
        cleaned = re.sub(pattern, '', matches[0], flags=re.M)
        cleaned = repair_json(cleaned)
    try:
        cleaned_jsons.append(json.loads(cleaned))
    except:
        print(f"Could not json loads for {idx}")
        cleaned_jsons.append("")
        continue

0
1
2
Could not json loads for 2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [119]:
data['cleaned_jsons'] = cleaned_jsons

In [120]:
data[['cleaned_jsons']]

,cleaned_jsons
0,"{'merchant_name': 'Walmart', 'country': 'Unite..."
1,"{'merchant_name': 'Amazon', 'country': 'United..."
2,
3,"{'merchant_name': 'Target', 'country': 'United..."
4,"{'merchant_name': 'The Home Depot', 'country':..."
...,...
93,"{'merchant_name': 'Shake Shack', 'country': 'U..."
94,"{'merchant_name': 'IKEA', 'country': 'Sweden, ..."
95,"{'merchant_name': 'Wayfair', 'country': 'Unite..."
96,"{'merchant_name': 'Williams-Sonoma', 'country'..."


In [126]:
normalized_df = pd.json_normalize(data['cleaned_jsons'])
normalized_df = normalized_df.dropna(how='all')
normalized_df = normalized_df.drop('website',axis=1)
normalized_df

,merchant_name,country,city,number_of_customers,daily_customers,yearly_revenue,valuation,years_in_business,founded_year,sector,num_employees
0,Walmart,United States,"Bentonville, Arkansas",,2 million (approximate),$569 billion,$407 billion (as of 2019),75,1962,Retail,"2,300,000"
1,Amazon,United States,Seattle,,More than 385 million worldwide per day (as of...,$469 billion USD (for the fiscal year ending i...,$1.7 trillion as of April 2021,27 years (founded in 1994),1994,"E-commerce, Cloud Computing, Consumer Electronics",NaN
3,Target,United States,"Minneapolis, Minnesota (headquarters)",,Estimated millions annually based on revenue a...,$125 billion as of 2020 according to Forbes,$248 billion (as of March 2023),61 years (founded in 1902),1902,"Retail, specifically discount department store...",NaN
4,The Home Depot,United States,,10 million customers per day,10 million customers per day,$125 billion in 2020 (from Investopedia),$320 billion as of September 2020 (from Forbes),49 years,1978,Consumer Discretionary - Home Improvement Retail,NaN
5,Kroger,United States,"Various - headquartered in Cincinnati, Ohio",,,$124 billion (as of 2020),$30 billion as of 2021,Over 135 years,1883,Retail - Grocery Stores,NaN
...,...,...,...,...,...,...,...,...,...,...,...
93,Shake Shack,United States,,,1 million+ (estimated),$450-500 million,$3.7 billion,20,2004,Foodservice,NaN
94,IKEA,"Sweden, Denmark, United States, Belgium, Russi...",,,2 million worldwide,$50 billion USD (as of 2018),$79.3 billion USD (as of September 2021),76 years,1943,"Furniture, home accessories, and kitchen appli...",NaN
95,Wayfair,United States,"Boston, Massachusetts",Not explicitly provided. Estimated based on ma...,Not explicitly provided. Noted that Wayfair se...,$14 billion USD in 2021,$67 billion as of October 2021,Over 25 years,1999,"E-commerce, specifically home goods and furnis...",NaN
96,Williams-Sonoma,United States,,,,$1.2 billion (approximate),"$8.5 billion (as of 2020, approximate)",76 years,1956,Retail - Home & Kitchen,NaN


In [128]:
normalized_df.columns

Index(['merchant_name', 'country', 'city', 'number_of_customers',
       'daily_customers', 'yearly_revenue', 'valuation', 'years_in_business',
       'founded_year', 'sector', 'num_employees'],
      dtype='object')

In [136]:
normalized_df.to_csv("attributes.csv",index=False)

In [129]:
from sentence_transformers import SentenceTransformer

/Users/vishaldawar/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [130]:
df = normalized_df.copy()

# Step 1: Convert each merchant row into a descriptive text
def row_to_text(row):
    return (
        f"{row['merchant_name']} is a company in the {row['sector']} sector, "
        f"based in {row['city']}, {row['country']}. "
        f"It has around {row['number_of_customers']} customers and {row['daily_customers']} daily customers. "
        f"The yearly revenue is {row['yearly_revenue']} with a valuation of {row['valuation']}. "
        f"Founded in {row['founded_year']}, it has been in business for {row['years_in_business']} years "
        f"and employs approximately {row['num_employees']} people."
    )

df["text"] = df.apply(row_to_text, axis=1)

In [132]:
model_name = "/Users/vishaldawar/Phi-3-mini-4k-instruct-fp16.gguf"

# Step 2: Load a compact and efficient embedding model (no API key needed)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Step 3: Generate embeddings
df["embedding"] = model.encode(df["text"].tolist(), show_progress_bar=True).tolist()

# Step 4: Inspect results
print(df[["merchant_name", "text"]].head())
print("\nEmbedding vector size:", len(df["embedding"][0]))

Batches: 100%|████████████████████████████████████| 4/4 [00:00<00:00,  5.30it/s]

    merchant_name                                               text
0         Walmart  Walmart is a company in the Retail sector, bas...
1          Amazon  Amazon is a company in the E-commerce, Cloud C...
3          Target  Target is a company in the Retail, specificall...
4  The Home Depot  The Home Depot is a company in the Consumer Di...
5          Kroger  Kroger is a company in the Retail - Grocery St...

Embedding vector size: 384


In [149]:
df

,merchant_name,country,city,number_of_customers,daily_customers,yearly_revenue,valuation,years_in_business,founded_year,sector,num_employees,text,embedding
0,Walmart,United States,"Bentonville, Arkansas",,2 million (approximate),$569 billion,$407 billion (as of 2019),75,1962,Retail,"2,300,000","Walmart is a company in the Retail sector, bas...","[0.1002039834856987, -0.04908445477485657, -0...."
1,Amazon,United States,Seattle,,More than 385 million worldwide per day (as of...,$469 billion USD (for the fiscal year ending i...,$1.7 trillion as of April 2021,27 years (founded in 1994),1994,"E-commerce, Cloud Computing, Consumer Electronics",NaN,"Amazon is a company in the E-commerce, Cloud C...","[0.09570392966270447, -0.07792823016643524, -0..."
3,Target,United States,"Minneapolis, Minnesota (headquarters)",,Estimated millions annually based on revenue a...,$125 billion as of 2020 according to Forbes,$248 billion (as of March 2023),61 years (founded in 1902),1902,"Retail, specifically discount department store...",NaN,"Target is a company in the Retail, specificall...","[0.050721243023872375, -0.03158675506711006, -..."
4,The Home Depot,United States,,10 million customers per day,10 million customers per day,$125 billion in 2020 (from Investopedia),$320 billion as of September 2020 (from Forbes),49 years,1978,Consumer Discretionary - Home Improvement Retail,NaN,The Home Depot is a company in the Consumer Di...,"[0.018078885972499847, -0.043807052075862885, ..."
5,Kroger,United States,"Various - headquartered in Cincinnati, Ohio",,,$124 billion (as of 2020),$30 billion as of 2021,Over 135 years,1883,Retail - Grocery Stores,NaN,Kroger is a company in the Retail - Grocery St...,"[0.0418100580573082, -0.054275721311569214, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,Shake Shack,United States,,,1 million+ (estimated),$450-500 million,$3.7 billion,20,2004,Foodservice,NaN,Shake Shack is a company in the Foodservice se...,"[-0.015121443197131157, -0.08053352683782578, ..."
94,IKEA,"Sweden, Denmark, United States, Belgium, Russi...",,,2 million worldwide,$50 billion USD (as of 2018),$79.3 billion USD (as of September 2021),76 years,1943,"Furniture, home accessories, and kitchen appli...",NaN,"IKEA is a company in the Furniture, home acces...","[-0.038097869604825974, -0.005186257418245077,..."
95,Wayfair,United States,"Boston, Massachusetts",Not explicitly provided. Estimated based on ma...,Not explicitly provided. Noted that Wayfair se...,$14 billion USD in 2021,$67 billion as of October 2021,Over 25 years,1999,"E-commerce, specifically home goods and furnis...",NaN,"Wayfair is a company in the E-commerce, specif...","[0.014679105021059513, -0.08961138874292374, -..."
96,Williams-Sonoma,United States,,,,$1.2 billion (approximate),"$8.5 billion (as of 2020, approximate)",76 years,1956,Retail - Home & Kitchen,NaN,Williams-Sonoma is a company in the Retail - H...,"[0.02870260179042816, 0.016919763758778572, -0..."


In [137]:
df.to_csv("./attributes_and_embeddings.csv",index=False)

In [147]:
for idx, row in data.iterrows():
    if 'uber' in row['merchant_name'].lower():
        print(row['merchant_name'])

Uber


In [151]:
from sklearn.metrics.pairwise import cosine_similarity
from typing import List

In [155]:
def findSimilarity(w1, w2):
    """
    Returns normalized levenshtein similarity between two input strings w1 and w2.
    """
    dot_product = np.dot(w1, w2)
    l2_w1 = np.linalg.norm(w1)
    l2_w2 = np.linalg.norm(w2)
    cosine_simi = dot_product / (l2_w1 * l2_w2)
    return cosine_simi

def match_customers(customer_name: str, embedding: List[float], data: pd.DataFrame):
    """
    Finds the top 3 similar job names based on the input query using levenshtein similarity w1 and w2.
    """

    all_simi = pd.DataFrame(columns=['input','match','score'])
    for idx, row in data.iterrows():
        w1 = customer_name
        for col in ['embedding']:
            w2 = row[col]
            simi = findSimilarity(embedding, w2)
            all_simi.loc[len(all_simi)] = [w1, row['merchant_name'], simi]
    matches = all_simi.sort_values('score',ascending=False).head(10)[['input','match','score']]
    return matches

In [156]:
customer_name = "Uber"
embedding = df[df['merchant_name'] == "Uber"]['embedding'].values[0]
match_customers(customer_name, embedding, df)

,input,match,score
28,Uber,Uber,1.000000
55,Uber,Tesla,0.551929
21,Uber,Google,0.513667
19,Uber,Apple,0.512867
20,Uber,Microsoft,0.493264
76,Uber,T-Mobile,0.483616
62,Uber,Chevron,0.475628
10,Uber,Starbucks,0.466973
1,Uber,Amazon,0.466578
56,Uber,Toyota,0.459506


In [4]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sentence_transformers import SentenceTransformer

In [72]:
real = pd.read_csv("real_merchants_info.csv")
real

,merchant_name,error,raw_response
0,Walmart,Parsing failed,"\nsolution: {\n ""merchant_name"": ""Walmart"",..."
1,Amazon,Parsing failed,"\nexplanation: {\n ""merchant_name"": ""Amazon..."
2,Costco,Parsing failed,"\n- bob: To provide accurate information, I'll..."
3,Target,Parsing failed,\n- [solution]: Since the provided search resu...
4,The Home Depot,Parsing failed,"\n**response:** {\n ""merchant_name"": ""The H..."
...,...,...,...
93,Shake Shack,Parsing failed,"\nresponse: {\n ""merchant_name"": ""Shake Sha..."
94,IKEA,Parsing failed,"\nOutput:\n{\n ""merchant_name"": ""IKEA"",\n ..."
95,Wayfair,Parsing failed,"\nTeacher: Given the available information, I'..."
96,Williams-Sonoma,Parsing failed,\n- [support]: Given the constraints of not ha...


In [74]:
import pandas as pd
import numpy as np
import random

# ---------------------------
# Base setup
# ---------------------------
np.random.seed(42)

merchant_list = [
    'Walmart', 'Amazon', 'Costco', 'Target', 'The Home Depot', 'Kroger', 'Lowes', 'CVS Health', 'Walgreens', 'Best Buy',
    'McDonalds', 'Starbucks', 'Subway', 'Chick-fil-A', 'Taco Bell', 'Burger King', 'Wendys', 'Dunkin', 'Chipotle', 'Dominos Pizza',
    'Apple', 'Microsoft', 'Google', 'eBay', 'PayPal', 'Etsy', 'Shopify', 'Netflix', 'Spotify', 'Uber',
    'American Airlines', 'Delta Air Lines', 'United Airlines', 'Southwest Airlines', 'Emirates', 'Lufthansa', 'Air France-KLM',
    'British Airways', 'Singapore Airlines', 'Cathay Pacific', 'Marriott International', 'Hilton', 'Hyatt', 'InterContinental Hotels',
    'Airbnb', 'Booking.com', 'Expedia', 'Nike', 'Adidas', 'H&M', 'Zara', 'Uniqlo', 'Gap Inc', 'Lululemon', 'TJX Companies',
    'Ross Stores', 'Tesla', 'Toyota', 'Ford', 'General Motors', 'Shell', 'ExxonMobil', 'BP', 'Chevron', 'Macys', 'Nordstrom',
    'Kohls', 'Saks Fifth Avenue', 'Neiman Marcus', 'Bloomingdales', 'Disney', 'Universal Parks', 'AMC Theatres', 'Regal Cinemas',
    'Ticketmaster', 'Verizon', 'AT&T', 'T-Mobile', 'Comcast', 'Charter Communications', 'Tesco', 'Carrefour', 'Aldi', 'Lidl',
    'Whole Foods', 'Trader Joes', 'Publix', 'Safeway', 'KFC', 'Pizza Hut', 'Papa Johns', 'Five Guys', 'Panera Bread', 'Shake Shack',
    'IKEA', 'Wayfair', 'Williams-Sonoma', 'Bed Bath & Beyond'
]

# ---------------------------
# MCC category mapping
# ---------------------------
mcc_mapping = {
    "Groceries": ['Walmart', 'Costco', 'Target', 'Kroger', 'Tesco', 'Carrefour', 'Aldi', 'Lidl', 'Whole Foods', 'Trader Joes', 'Publix', 'Safeway'],
    "E-commerce": ['Amazon', 'eBay', 'Shopify', 'Etsy', 'Wayfair'],
    "Restaurants": ['McDonalds', 'Starbucks', 'Subway', 'Chick-fil-A', 'Taco Bell', 'Burger King', 'Wendys', 'Dunkin', 'Chipotle', 'Dominos Pizza', 'KFC', 'Pizza Hut', 'Papa Johns', 'Five Guys', 'Panera Bread', 'Shake Shack'],
    "Technology": ['Apple', 'Microsoft', 'Google', 'PayPal'],
    "Streaming": ['Netflix', 'Spotify', 'Disney', 'Universal Parks', 'AMC Theatres', 'Regal Cinemas'],
    "Travel": ['American Airlines', 'Delta Air Lines', 'United Airlines', 'Southwest Airlines', 'Emirates', 'Lufthansa', 'Air France-KLM', 'British Airways', 'Singapore Airlines', 'Cathay Pacific'],
    "Hotels": ['Marriott International', 'Hilton', 'Hyatt', 'InterContinental Hotels', 'Airbnb', 'Booking.com', 'Expedia'],
    "Retail": ['Nike', 'Adidas', 'H&M', 'Zara', 'Uniqlo', 'Gap Inc', 'Lululemon', 'TJX Companies', 'Ross Stores', 'Macys', 'Nordstrom', 'Kohls', 'Saks Fifth Avenue', 'Neiman Marcus', 'Bloomingdales', 'IKEA', 'Williams-Sonoma', 'Bed Bath & Beyond'],
    "Automotive": ['Tesla', 'Toyota', 'Ford', 'General Motors'],
    "Energy": ['Shell', 'ExxonMobil', 'BP', 'Chevron'],
    "Telecom": ['Verizon', 'AT&T', 'T-Mobile', 'Comcast', 'Charter Communications'],
    "Ticketing": ['Ticketmaster']
}

# Reverse mapping merchant → MCC
merchant_to_mcc = {m: k for k, v in mcc_mapping.items() for m in v}

# Country mapping (simplified)
merchant_to_country = {
    "Carrefour": "FRA", "Tesco": "GBR", "Lidl": "DEU", "Aldi": "DEU", "Emirates": "ARE",
    "Lufthansa": "DEU", "Air France-KLM": "FRA", "British Airways": "GBR",
    "Singapore Airlines": "SGP", "Cathay Pacific": "HKG", "Toyota": "JPN", "BP": "GBR"
}
default_country = "USA"

# ---------------------------
# Synthetic generation
# ---------------------------

num_cards = 500
num_transactions = 5000  # 👈 change this to generate more or fewer transactions

cards = [f"card_{i}" for i in range(1, num_cards + 1)]

data = []
for _ in range(num_transactions):
    merchant = random.choice(merchant_list)
    mcc = merchant_to_mcc.get(merchant, "Retail")
    card = random.choice(cards)
    country = merchant_to_country.get(merchant, default_country)

    # amount distribution varies by category
    if mcc in ["Streaming", "E-commerce"]:
        amount = round(np.random.uniform(5, 50), 2)
    elif mcc in ["Groceries", "Restaurants"]:
        amount = round(np.random.uniform(10, 300), 2)
    elif mcc in ["Hotels", "Travel"]:
        amount = round(np.random.uniform(50, 1500), 2)
    elif mcc in ["Technology", "Automotive"]:
        amount = round(np.random.uniform(100, 5000), 2)
    elif mcc in ["Energy"]:
        amount = round(np.random.uniform(30, 200), 2)
    else:
        amount = round(np.random.uniform(5, 1000), 2)

    data.append([card, country, merchant, amount, mcc])

transactions = pd.DataFrame(data, columns=["card_number", "country_code", "merchant", "amount", "mcc"])

# Display sample
print(transactions.head())
print("\nTotal transactions:", len(transactions))


  card_number country_code        merchant  amount          mcc
0    card_248          HKG  Cathay Pacific  593.08       Travel
1    card_184          FRA       Carrefour  285.71    Groceries
2    card_400          USA         Walmart  222.28    Groceries
3    card_141          USA    Panera Bread  183.61  Restaurants
4     card_20          USA          Airbnb  276.23       Hotels

Total transactions: 5000


In [75]:
transactions.to_csv("all_transactions.csv",index=False)

In [5]:
# Example dataset of transactions
transactions = pd.DataFrame({
    "card_number": ["card_1", "card_1", "card_2", "card_3", "card_3"],
    "country_code": ["USA", "USA", "FRA", "IND", "IND"],
    "merchant": ["Netflix", "Walmart", "Carrefour", "Amazon", "Big Bazaar"],
    "amount": [15.99, 220.50, 80.20, 55.30, 60.75],
    "mcc": ["Streaming", "Groceries", "Retail", "E-commerce", "Groceries"]
})

# Step 1: Convert each transaction into a text description
transactions["text"] = transactions.apply(
    lambda r: f"Transaction of ${r.amount} at {r.merchant} in {r.country_code} for {r.mcc}",
    axis=1
)

In [6]:
transactions

,card_number,country_code,merchant,amount,mcc,text
0,card_1,USA,Netflix,15.99,Streaming,Transaction of $15.99 at Netflix in USA for St...
1,card_1,USA,Walmart,220.50,Groceries,Transaction of $220.5 at Walmart in USA for Gr...
2,card_2,FRA,Carrefour,80.20,Retail,Transaction of $80.2 at Carrefour in FRA for R...
3,card_3,IND,Amazon,55.30,E-commerce,Transaction of $55.3 at Amazon in IND for E-co...
4,card_3,IND,Big Bazaar,60.75,Groceries,Transaction of $60.75 at Big Bazaar in IND for...


In [16]:
txns = transactions.drop(['text'],axis=1)

In [8]:
# Step 2: Get embeddings
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(transactions["text"].tolist())

# Step 3: Cluster embeddings to find pseudo-classes (behavioral clusters)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(embeddings)
kmeans = KMeans(n_clusters=3, random_state=42)
pseudo_labels = kmeans.fit_predict(X_scaled)

In [11]:
pseudo_labels

array([2, 0, 1, 1, 0], dtype=int32)

In [13]:
cluster_data

array([[-4.57453191e-01,  3.08506399e-01,  9.72278595e-01,
         5.68620443e-01, -1.14189482e+00, -5.88826872e-02,
        -5.60658835e-02, -4.57121015e-01, -1.54503834e+00,
        -1.54029107e+00, -5.65088987e-01,  7.86212444e-01,
        -1.47883058e-01, -6.73111081e-01,  1.20970771e-01,
        -2.65956998e-01,  6.29441321e-01,  6.00432813e-01,
        -5.73237240e-01, -8.09952557e-01,  1.04243422e+00,
         4.39816952e-01, -1.07020998e+00,  1.75650969e-01,
        -1.01497963e-01, -1.08456051e+00, -1.78263158e-01,
        -9.73463953e-01,  1.30614236e-01,  1.63027358e+00,
        -1.48556662e+00, -1.57260013e+00, -2.08288416e-01,
         6.48234904e-01, -1.08082521e+00,  1.19435692e+00,
         1.37674081e+00,  8.38240981e-01, -1.06325078e+00,
         6.08519912e-02,  9.11638260e-01,  1.05712163e+00,
        -6.60398662e-01, -6.82304204e-01, -7.85366952e-01,
         2.56119877e-01, -6.82736933e-01,  6.70981184e-02,
        -1.36424685e+00,  4.14292246e-01,  1.06050229e+0

In [14]:
# Step 4: Synthetic oversampling (custom interpolation)
synthetic_points = []

for cluster_id in np.unique(pseudo_labels):
    cluster_data = X_scaled[pseudo_labels == cluster_id]
    n = len(cluster_data)
    if n < 2:
        print(f"Skipping cluster {cluster_id} (only {n} sample).")
        continue

    n_new = n  # number of synthetic samples to add = cluster size
    idx1 = np.random.randint(0, n, n_new)
    idx2 = np.random.randint(0, n, n_new)
    lam = np.random.rand(n_new, 1)

    # Linear interpolation (similar to SMOTE but unsupervised)
    synth = cluster_data[idx1] * lam + cluster_data[idx2] * (1 - lam)
    synthetic_points.append(synth)

# Combine
synthetic_data = np.vstack(synthetic_points)
synthetic_df = pd.DataFrame(scaler.inverse_transform(synthetic_data), columns=[f"dim_{i}" for i in range(synthetic_data.shape[1])])

print(f"Original samples: {len(transactions)}")
print(f"Synthetic samples added: {len(synthetic_df)}")

Skipping cluster 2 (only 1 sample).
Original samples: 5
Synthetic samples added: 4


In [15]:
synthetic_df

,dim_0,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,...,dim_374,dim_375,dim_376,dim_377,dim_378,dim_379,dim_380,dim_381,dim_382,dim_383
0,0.038548,0.037965,0.009196,0.013508,-0.031804,-0.023765,0.010994,-0.025609,0.018822,0.023051,...,-0.011319,-0.021008,-0.024675,-0.070620,-0.054723,0.068287,0.059204,-0.065720,-0.032245,-0.013063
1,0.051037,0.077542,-0.044711,0.005778,-0.032453,-0.030024,-0.012374,0.019534,0.042759,0.048203,...,0.005237,-0.015962,-0.008759,-0.062103,-0.042108,-0.000446,0.035886,-0.100742,-0.029990,-0.030972
2,0.046081,-0.004711,-0.030309,-0.003141,0.011967,-0.029431,-0.001941,-0.063982,0.086089,0.035813,...,0.007730,0.007024,0.036800,-0.052031,-0.015176,0.028738,0.041142,-0.148110,-0.041254,0.034706
3,0.046081,-0.004711,-0.030309,-0.003141,0.011967,-0.029431,-0.001941,-0.063982,0.086089,0.035813,...,0.007730,0.007024,0.036800,-0.052031,-0.015176,0.028738,0.041142,-0.148110,-0.041254,0.034706


In [18]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer

# Assume you already have:
# - transactions (original dataframe)
# - synthetic_data (your synthetic embeddings)
# - embeddings (original embeddings corresponding to each transaction)

synthetic_data = synthetic_df.copy()

# 1️⃣ Prepare real embeddings and scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(embeddings)
synthetic_scaled = scaler.transform(synthetic_data)

# 2️⃣ Find nearest neighbors
nn = NearestNeighbors(n_neighbors=1, metric='cosine')
nn.fit(X_scaled)
distances, indices = nn.kneighbors(synthetic_scaled)

# 3️⃣ For each synthetic embedding, pick the closest real transaction
synthetic_like = txns.iloc[indices.flatten()].copy().reset_index(drop=True)

# 4️⃣ Optionally, perturb numeric fields slightly to make them unique
numeric_cols = ['amount']
for col in numeric_cols:
    if col in synthetic_like.columns:
        noise = np.random.normal(0, 0.05, len(synthetic_like)) * synthetic_like[col]
        synthetic_like[col] = (synthetic_like[col] + noise).round(2)

# 5️⃣ Combine original + synthetic transactions
txns['type'] = 'original'
synthetic_like['type'] = 'synthetic'
augmented_df = pd.concat([txns, synthetic_like], ignore_index=True)

print(f"Original data: {len(txns)} rows")
print(f"Augmented data: {len(augmented_df)} rows")
augmented_df.head(10)


Original data: 5 rows
Augmented data: 9 rows


,card_number,country_code,merchant,amount,mcc,type
0,card_1,USA,Netflix,15.99,Streaming,original
1,card_1,USA,Walmart,220.50,Groceries,original
2,card_2,FRA,Carrefour,80.20,Retail,original
3,card_3,IND,Amazon,55.30,E-commerce,original
4,card_3,IND,Big Bazaar,60.75,Groceries,original
5,card_1,USA,Walmart,222.43,Groceries,synthetic
6,card_3,IND,Big Bazaar,61.74,Groceries,synthetic
7,card_3,IND,Amazon,56.02,E-commerce,synthetic
8,card_3,IND,Amazon,54.55,E-commerce,synthetic


In [40]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer

def generate_synthetic_merchants(
    df: pd.DataFrame,
    n_synthetic: int = None,
    oversample_factor: float = None,
    model_name: str = "sentence-transformers/all-MiniLM-L6-v2"
):
    """
    Generate realistic synthetic merchant/transaction data using embeddings + nearest neighbors.

    Parameters:
    - df: Original DataFrame
    - n_synthetic: Total number of synthetic samples to generate (optional)
    - oversample_factor: Multiplier for oversampling (optional)
    - model_name: HuggingFace sentence transformer to embed text fields

    Returns:
    - augmented_df: pd.DataFrame (original + synthetic)
    """
    # --- Determine oversampling size
    if n_synthetic is None and oversample_factor is None:
        raise ValueError("Please specify either n_synthetic or oversample_factor")
    if oversample_factor is not None:
        n_synthetic = int(len(df) * oversample_factor)

    # --- Auto-detect text & numeric columns
    text_cols = df.select_dtypes(include=['object']).columns.tolist()
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

    print(f"🧠 Text columns: {text_cols}")
    print(f"🔢 Numeric columns: {numeric_cols}")
    print(f"📈 Generating {n_synthetic} synthetic samples...")

    # --- Combine text columns into semantic text for embedding
    text_data = df[text_cols].astype(str).agg(" ".join, axis=1)

    # --- Generate embeddings
    model = SentenceTransformer(model_name)
    embeddings = model.encode(text_data, show_progress_bar=True)

    # --- Scale embeddings
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(embeddings)

    # --- Generate new synthetic embeddings (random interpolation)
    np.random.seed(42)
    idx1 = np.random.randint(0, len(embeddings), n_synthetic)
    idx2 = np.random.randint(0, len(embeddings), n_synthetic)
    lam = np.random.rand(n_synthetic, 1)
    synthetic_emb = lam * embeddings[idx1] + (1 - lam) * embeddings[idx2]
    synthetic_scaled = scaler.transform(synthetic_emb)

    # --- Match to nearest neighbors in original space
    nn = NearestNeighbors(n_neighbors=1, metric='cosine')
    nn.fit(X_scaled)
    distances, indices = nn.kneighbors(synthetic_scaled)

    synthetic_like = df.iloc[indices.flatten()].copy().reset_index(drop=True)

    # --- Add slight random noise to numeric features
    for col in numeric_cols:
        noise = np.random.normal(0, 0.05, len(synthetic_like))
        synthetic_like[col] = synthetic_like[col] + (synthetic_like[col] * noise)
        if np.issubdtype(df[col].dtype, np.integer):
            synthetic_like[col] = synthetic_like[col].round().astype(int)
        else:
            synthetic_like[col] = synthetic_like[col].round(2)

    # --- Optional: add slight randomness to merchant_name / city for uniqueness
    synthetic_like["merchant"] = synthetic_like["merchant"] + "_syn"
    # synthetic_like["city"] = synthetic_like["city"] + "_syn"

    # --- Combine both
    df['type'] = 'real'
    synthetic_like['type'] = 'synthetic'
    augmented_df = pd.concat([df, synthetic_like], ignore_index=True)
    return augmented_df, text_data


In [45]:
# Assume df = your transactions dataframe
if 'type' in txns.columns:
    txns = txns.drop('type',axis=1)
    
augmented_df, text_data = generate_synthetic_merchants(
    df=txns,
    n_synthetic = 1000
)

🧠 Text columns: ['card_number', 'country_code', 'merchant', 'mcc']
🔢 Numeric columns: ['amount']
📈 Generating 1000 synthetic samples...


Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00, 96.11it/s]


In [46]:
text_data

0       card_1 USA Netflix Streaming
1       card_1 USA Walmart Groceries
2        card_2 FRA Carrefour Retail
3       card_3 IND Amazon E-commerce
4    card_3 IND Big Bazaar Groceries
dtype: object

In [44]:
txns

,card_number,country_code,merchant,amount,mcc,type
0,card_1,USA,Netflix,15.99,Streaming,real
1,card_1,USA,Walmart,220.50,Groceries,real
2,card_2,FRA,Carrefour,80.20,Retail,real
3,card_3,IND,Amazon,55.30,E-commerce,real
4,card_3,IND,Big Bazaar,60.75,Groceries,real
